In [1]:
import data_parser.DataParser as dp
import turicreate as tc

In [2]:
filepaths = {
    'articles': '../../data/articles.csv',
    'customers': '../../data/customers.csv',
    'transactions': '../../data/transactions_train.csv'
}

In [13]:
parser = dp.DataParser(filepaths)

articles = parser.get_data('articles')
customers = parser.get_data('customers')

In [ ]:
# articles = tc.SFrame(data=articles.to_dict(orient='list'))
# customers = tc.SFrame(data=customers.to_dict(orient='list'))

In [ ]:
# articles_similarity_model = tc.recommender.item_content_recommender.create(articles, item_id='article_id', max_item_neighborhood_size=32, similarity_metrics='cosine')
# articles_similarity_matrix = articles_similarity_model.get_similar_items(articles['article_id'], 12)

In [ ]:
# articles_similarity_matrix.print_rows(60)

In [4]:
transactions = parser.get_data('transactions')

In [6]:
train = transactions.groupby(['customer_id', 'article_id'])[['t_dat']].count().reset_index().rename(columns={'t_dat': 'purchase_count'})
train.head()

,customer_id,article_id,purchase_count
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,176209023,1
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601006,2
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601043,1
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,607642008,1
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,625548001,1


In [7]:
train = train.merge(train.groupby(['customer_id'])['purchase_count'].sum().reset_index().rename(columns={'purchase_count': 'total'}), how='outer', on='customer_id')

In [8]:
train.head()

,customer_id,article_id,purchase_count,total
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,176209023,1,21
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601006,2,21
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601043,1,21
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,607642008,1,21
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,625548001,1,21


In [9]:
train['weights'] = train['purchase_count'] / train['total']
train.sort_values('total')

,customer_id,article_id,purchase_count,total,weights
27306438,ffffd9ac14e89946416d80e791d064701994755c3ab686...,806050001,1,1,1.000000
1695572,0fd922bfb7592b8eb1b85e1e57beee2a3137dc1b23f36a...,658030005,1,1,1.000000
24394616,e4b954cb788b57047e524c61dd843c8eb30794dfdb361a...,532102015,1,1,1.000000
21115644,c5cff6a4c550841a290cfd0bf0b812d16bd052962012b6...,740213001,1,1,1.000000
7001423,41b7bbda95d76c6d255a65da46c6199a21af84b7ee5d53...,625518003,1,1,1.000000
...,...,...,...,...,...
20302423,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,670162002,3,1895,0.001583
20302424,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,670279001,1,1895,0.000528
20302425,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,670655001,1,1895,0.000528
20302419,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,667870001,1,1895,0.000528


In [10]:
import pandas as pd

sub = pd.read_csv('./data/sample_submission.csv')

sub.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


In [12]:
len(sub['customer_id'].unique())

1371980

In [14]:
len(customers['customer_id'].unique())

1371980

In [15]:
len(transactions['customer_id'].unique())

1362281